In [1]:
import os 
cwd = os.getcwd()
print(cwd)

C:\Users\anish\OneDrive\Documents\GitHub\data-analytics-PYTHON


In [50]:
import os
import json
import pandas as pd
import re
from tabulate import tabulate

# File paths
main_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder"
csv_file = os.path.join(main_folder, "sales_data.csv")
json_folder = os.path.join(main_folder, "product_details")
txt_folder = os.path.join(main_folder, "product_descriptions")

def load_sales_data(csv_file):
    """Load sales data from a CSV file and return as a DataFrame."""
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        if "Product_SKU" not in df.columns:
            print("❌ Error: 'Product_SKU' column not found in CSV!")
            return pd.DataFrame()
        df.fillna(0, inplace=True)
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
        return df
    else:
        print("❌ Error: CSV file not found!")
        return pd.DataFrame()

def load_product_details(json_folder):
    """Load product details from JSON files."""
    product_details = {}
    if os.path.exists(json_folder):
        for filename in os.listdir(json_folder):
            if filename.startswith("details_") and filename.endswith('.json'):
                sku = filename.replace("details_", "").replace('.json', '')
                with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as file:
                    product_details[sku] = json.load(file)
    return product_details

def load_product_descriptions(txt_folder):
    """Load product descriptions from text files."""
    product_descriptions = {}
    if os.path.exists(txt_folder):
        for filename in os.listdir(txt_folder):
            if filename.startswith("description_") and filename.endswith('.txt'):
                sku = filename.replace("description_", "").replace('.txt', '')
                with open(os.path.join(txt_folder, filename), 'r', encoding='utf-8') as file:
                    product_descriptions[sku] = file.read().strip()
    return product_descriptions

def validate_sku(sku):
    """Validate SKU: It should be exactly 13 characters, alphanumeric, and uppercase."""
    return bool(re.fullmatch(r"[A-Z0-9]{13}", sku))
    
def update_sales_data(sku, sales_values):
    """Update or add sales data for a product SKU."""
    global sales_data
    if sku in sales_data["Product_SKU"].values:
        for i, val in enumerate(sales_values):
            sales_data.loc[sales_data["Product_SKU"] == sku, f"Day{i+1}"] = val
    else:
        new_row = {"Product_SKU": sku, **{f"Day{i+1}": sales_values[i] for i in range(14)}}
        sales_data = pd.concat([sales_data, pd.DataFrame([new_row])], ignore_index=True)
    
    print("✅ Sales data updated successfully!")



def update_product_details(sku, name, brand, model, specs, price, availability):
    """Update or add product details."""
    product_details[sku] = {
        "Product Name": name,
        "Brand": brand,
        "Model": model,
        "Specifications": specs,
        "Price": price,
        "Availability": availability
    }
    print("✅ Product details updated successfully!")

def update_product_description(sku, description):
    """Update or add product description."""
    product_descriptions[sku] = description
    print("✅ Product description updated successfully!")

def update():
    """Collect input and update all records."""
    sku = input("Enter Product SKU: ").strip().upper()
    if not validate_sku(sku):
        print("❌ Invalid SKU! It should be exactly 13 characters, alphanumeric, and uppercase.")
        return
    
    sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
    sales_values = list(map(int, sales_data_input.split()))
    if len(sales_values) != 14:
        print("❌ Sales data must contain exactly 14 values.")
        return
    
    name = input("Enter Product Name: ")
    brand = input("Enter Brand: ")
    model = input("Enter Model: ")
    specs = input("Enter Specifications: ")
    price = input("Enter Price: ")
    availability = input("Enter Availability: ")
    description = input("Enter Product Description: ")
    
    update_sales_data(sku, sales_values)
    update_product_details(sku, name, brand, model, specs, price, availability)
    update_product_description(sku, description)
    print("✅ All product details updated successfully!")

def dump_data():
    """Dump updated data into files."""
    os.makedirs(main_folder, exist_ok=True)
    sales_data.to_csv(csv_file, index=False)
    print("✅ Sales data successfully written to CSV.")
    
    os.makedirs(json_folder, exist_ok=True)
    os.makedirs(txt_folder, exist_ok=True)
    
    for sku, details in product_details.items():
        with open(os.path.join(json_folder, f"details_{sku}.json"), "w", encoding="utf-8") as json_file:
            json.dump(details, json_file, indent=4)
    print("✅ Product details successfully written to JSON files.")
    
    for sku, description in product_descriptions.items():
        with open(os.path.join(txt_folder, f"description_{sku}.txt"), "w", encoding="utf-8") as txt_file:
            txt_file.write(description)
    print("✅ Product descriptions successfully written to text files.")

# Load initial data
sales_data = load_sales_data(csv_file)
product_details = load_product_details(json_folder)
product_descriptions = load_product_descriptions(txt_folder)
print("✅ Data Loaded Successfully")

# Example of updating with given data
update_sales_data("CMWKCILOP27KF", [8, 14, 16, 7, 15, 21, 14, 16, 32, 29, 26, 30, 25, 22])
update_product_details("CMWKCILOP27KF", "Pokemon Card", "GameFreak", "ScarletViolet151", "Genuine, TCG, English", "$1.99", "In stock")
update_product_description("CMWKCILOP27KF", "Original Pokemon TCG Pikachu card")

dump_data()  

✅ Data Loaded Successfully
✅ Sales data updated successfully!
✅ Product details updated successfully!
✅ Product description updated successfully!
✅ Sales data successfully written to CSV.
✅ Product details successfully written to JSON files.
✅ Product descriptions successfully written to text files.


In [58]:
def main():
    # Step 1: Load Data
    global sales_data, product_details, product_descriptions
    sales_data = load_sales_data(csv_file)
    product_details = load_product_details(json_folder)
    product_descriptions = load_product_descriptions(txt_folder)
    
    #  Update Data
    up = input("\n🔄 Do you want to update or add the data? (yes/no): ").strip().lower()
    if up == "yes":
        update()
        print("\n✅ Update Successful!")

        #  Dump Data
        dump_data()
        print("\n✅ Data Successfully Saved!")

        #  Reload and Show Final Data
        sales_data = load_sales_data(csv_file)
        product_details = load_product_details(json_folder)
        product_descriptions = load_product_descriptions(txt_folder)

        print("\n✅ Final Updated Data:\n")
        print("\n📊 Updated Sales Data:\n", tabulate(sales_data, headers="keys", tablefmt="grid"))
        print("\n📄 Updated Product Details:", json.dumps(product_details, indent=4))
        print("\n📝 Updated Product Descriptions:", json.dumps(product_descriptions, indent=4))
    
    else:
        print("\n🙏 Thank You! No changes were made.")

# Run the main function
main()



🔄 Do you want to update or add the data? (yes/no):  no



🙏 Thank You! No changes were made.


### **Step-by-Step Summary for Implementing the Project**  

---

### **1️⃣ Set Up the Environment**  
✅ **Import necessary Python packages**  
- `os` → To navigate files and directories  
- `json` → To work with JSON files  
- `csv` → To read and write CSV files  

✅ **Ensure files are loaded correctly**  
- If using **Google Colab**, upload files manually or mount Google Drive  
- If using **Jupyter Notebook**, ensure files are in the correct directory  

---

### **2️⃣ Load the Data**  
Create a function `load_data()` to:  
- Load sales data from `sales_data.csv` into a dictionary (`sales_data`)  
- Load product details from JSON files in the `product_details` folder into a dictionary (`product_details`)  
- Load product descriptions from text files in the `product_descriptions` folder into a dictionary (`product_descriptions`)  

---

### **3️⃣ Add/Update Product Information**  
Create functions to allow the admin to add or update data:

✅ **`update_sales_data()`**  
- Add/update **sales data** for a given SKU  

✅ **`update_product_details()`**  
- Add/update **product details** for a given SKU  

✅ **`update_product_description()`**  
- Add/update **product description** for a given SKU  

✅ **`update()`** (Main function to update all data)  
- Take **user input** for SKU, sales data, product details, and description  
- **Validate inputs**  
  - SKU must be **exactly 13 characters**  
  - Sales data must have **14 whole numbers**  
  - Product details and description are required  
- If inputs are **valid**, update all three dictionaries  
- Print a **success message**  
- Return updated dictionaries  

---

### **4️⃣ Overwrite and Save Data to Files**  
Create `dump_data()` function to:  
- **Write `sales_data` to `sales_data.csv`**  
- **Save `product_details` as JSON files** in the `product_details` folder  
- **Save `product_descriptions` as text files** in the `product_descriptions` folder  
- If folders **don’t exist, create them automatically**  

---

### **5️⃣ Final Execution**  
✅ Load existing data  
✅ Call `update()` to modify product info  
✅ Call `dump_data()` to save changes to files  

---

### **Outcome**  
📌 **Admin can:**
- Load product data ✅  
- Add new products ✅  
- Update existing product information ✅  
- Ensure data integrity ✅  
- Overwrite files safely ✅  